In [1]:
import sklearn
import os
import re
import csv
import pandas as pd
import numpy as mp
import seaborn as sns
import matplotlib.pyplot as plt
import neattext.functions as nfx
from bs4 import BeautifulSoup
import torch
from torch.cuda.amp import autocast
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW, DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification

import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

2.6.0+cu124
12.4
True


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 3
LEARNING_RATE = 2e-5

#mytokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
mytokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#mytokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [44]:
df = pd.read_pickle("traindataset.pkl")

In [6]:
#df = df.rename(columns={'labels': 'emotions'})

In [7]:
pd.set_option("display.max_columns", None)

In [8]:
df.head(20)

,text,emotions,label_vector,Clean_Text
0,"""If you don't wear BROWN AND ORANGE...YOU DON...","[anger, neutral]","[1, 0, 0, 0, 0, 0, 1]",If you dont wear BROWN AND ORANGEYOU DONT MAT...
1,"""What do Scottish people look like?"" How I wo...",[happiness],"[0, 0, 0, 1, 0, 0, 0]",What do Scottish people look like How I would...
2,"### A surprise, to be sure, but a welcome one",[surprise],"[0, 0, 0, 0, 0, 1, 0]",### A surprise to be sure but a welcome one
3,"'*Pray*, v. To ask that the laws of the unive...",[neutral],"[0, 0, 0, 0, 0, 0, 1]",*Pray* v To ask that the laws of the universe...
4,">it'll get invaded by tankie, unfortunately. ...",[neutral],"[0, 0, 0, 0, 0, 0, 1]",>itll get invaded by tankie unfortunately How...
5,>sexuality shouldn’t be a grouping category I...,"[anger, neutral]","[1, 0, 0, 0, 0, 0, 1]",>sexuality shouldn’t be a grouping category I...
6,And not all children's hospitals need the sam...,"[happiness, neutral]","[0, 0, 0, 1, 0, 0, 1]",And not all childrens hospitals need the same...
7,"Best number! [NAME], [NAME], [NAME]","[happiness, neutral]","[0, 0, 0, 1, 0, 0, 1]",Best number [NAME] [NAME] [NAME]
8,Calm down and relax are the worst things to s...,"[disgust, happiness, anger]","[1, 1, 0, 1, 0, 0, 0]",Calm down and relax are the worst things to s...
9,"Change is hard. Find comfort in victory, even...",[neutral],"[0, 0, 0, 0, 0, 0, 1]",Change is hard Find comfort in victory even t...


In [9]:
df

,text,emotions,label_vector,Clean_Text
0,"""If you don't wear BROWN AND ORANGE...YOU DON...","[anger, neutral]","[1, 0, 0, 0, 0, 0, 1]",If you dont wear BROWN AND ORANGEYOU DONT MAT...
1,"""What do Scottish people look like?"" How I wo...",[happiness],"[0, 0, 0, 1, 0, 0, 0]",What do Scottish people look like How I would...
2,"### A surprise, to be sure, but a welcome one",[surprise],"[0, 0, 0, 0, 0, 1, 0]",### A surprise to be sure but a welcome one
3,"'*Pray*, v. To ask that the laws of the unive...",[neutral],"[0, 0, 0, 0, 0, 0, 1]",*Pray* v To ask that the laws of the universe...
4,">it'll get invaded by tankie, unfortunately. ...",[neutral],"[0, 0, 0, 0, 0, 0, 1]",>itll get invaded by tankie unfortunately How...
...,...,...,...,...
48674,🤷🏻‍♀️ As a wise man once said: he was a bastar...,"[anger, neutral]","[1, 0, 0, 0, 0, 0, 1]",🤷‍ As a wise man once said: he was a bastard i...
48675,🤷🏼‍♀️ I was wondering the same thing. It looke...,"[happiness, surprise]","[0, 0, 0, 1, 0, 1, 0]",🤷‍ I was wondering the same thing It looked al...
48676,🦀MY BABYS DEAD🦀,"[sadness, neutral]","[0, 0, 0, 0, 1, 0, 1]",🦀MY BABYS DEAD🦀
48677,🦀🦀🦀🦀🦀🦀🦀🦀🦀I’m bad at this game🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀,"[anger, sadness]","[1, 0, 0, 0, 1, 0, 0]",🦀🦀🦀🦀🦀🦀🦀🦀🦀I’m bad at this game🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀


### aaa

In [10]:
pd.set_option('display.max_colwidth', 150)

In [11]:
df

,text,emotions,label_vector,Clean_Text
0,"""If you don't wear BROWN AND ORANGE...YOU DON'T MATTER!"" We need a tshirt with that on it asap!","[anger, neutral]","[1, 0, 0, 0, 0, 0, 1]",If you dont wear BROWN AND ORANGEYOU DONT MATTER We need a tshirt with that on it asap
1,"""What do Scottish people look like?"" How I would love to have been there to take a swing at that softball.",[happiness],"[0, 0, 0, 1, 0, 0, 0]",What do Scottish people look like How I would love to have been there to take a swing at that softball
2,"### A surprise, to be sure, but a welcome one",[surprise],"[0, 0, 0, 0, 0, 1, 0]",### A surprise to be sure but a welcome one
3,"'*Pray*, v. To ask that the laws of the universe be annulled in behalf of a single petitioner, confessedly unworthy.' — [NAME]",[neutral],"[0, 0, 0, 0, 0, 0, 1]",*Pray* v To ask that the laws of the universe be annulled in behalf of a single petitioner confessedly unworthy — [NAME]
4,">it'll get invaded by tankie, unfortunately. How apropos.",[neutral],"[0, 0, 0, 0, 0, 0, 1]",>itll get invaded by tankie unfortunately How apropos
...,...,...,...,...
48674,🤷🏻‍♀️ As a wise man once said: he was a bastard in life thus a bastard in death,"[anger, neutral]","[1, 0, 0, 0, 0, 0, 1]",🤷‍ As a wise man once said: he was a bastard in life thus a bastard in death
48675,🤷🏼‍♀️ I was wondering the same thing. It looked almost like a teriyaki.,"[happiness, surprise]","[0, 0, 0, 1, 0, 1, 0]",🤷‍ I was wondering the same thing It looked almost like a teriyaki
48676,🦀MY BABYS DEAD🦀,"[sadness, neutral]","[0, 0, 0, 0, 1, 0, 1]",🦀MY BABYS DEAD🦀
48677,🦀🦀🦀🦀🦀🦀🦀🦀🦀I’m bad at this game🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀,"[anger, sadness]","[1, 0, 0, 0, 1, 0, 0]",🦀🦀🦀🦀🦀🦀🦀🦀🦀I’m bad at this game🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀


In [12]:
df[df['emotions'].apply(lambda x: len(x) <= 3)]

,text,emotions,label_vector,Clean_Text
0,"""If you don't wear BROWN AND ORANGE...YOU DON'T MATTER!"" We need a tshirt with that on it asap!","[anger, neutral]","[1, 0, 0, 0, 0, 0, 1]",If you dont wear BROWN AND ORANGEYOU DONT MATTER We need a tshirt with that on it asap
1,"""What do Scottish people look like?"" How I would love to have been there to take a swing at that softball.",[happiness],"[0, 0, 0, 1, 0, 0, 0]",What do Scottish people look like How I would love to have been there to take a swing at that softball
2,"### A surprise, to be sure, but a welcome one",[surprise],"[0, 0, 0, 0, 0, 1, 0]",### A surprise to be sure but a welcome one
3,"'*Pray*, v. To ask that the laws of the universe be annulled in behalf of a single petitioner, confessedly unworthy.' — [NAME]",[neutral],"[0, 0, 0, 0, 0, 0, 1]",*Pray* v To ask that the laws of the universe be annulled in behalf of a single petitioner confessedly unworthy — [NAME]
4,">it'll get invaded by tankie, unfortunately. How apropos.",[neutral],"[0, 0, 0, 0, 0, 0, 1]",>itll get invaded by tankie unfortunately How apropos
...,...,...,...,...
48674,🤷🏻‍♀️ As a wise man once said: he was a bastard in life thus a bastard in death,"[anger, neutral]","[1, 0, 0, 0, 0, 0, 1]",🤷‍ As a wise man once said: he was a bastard in life thus a bastard in death
48675,🤷🏼‍♀️ I was wondering the same thing. It looked almost like a teriyaki.,"[happiness, surprise]","[0, 0, 0, 1, 0, 1, 0]",🤷‍ I was wondering the same thing It looked almost like a teriyaki
48676,🦀MY BABYS DEAD🦀,"[sadness, neutral]","[0, 0, 0, 0, 1, 0, 1]",🦀MY BABYS DEAD🦀
48677,🦀🦀🦀🦀🦀🦀🦀🦀🦀I’m bad at this game🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀,"[anger, sadness]","[1, 0, 0, 0, 1, 0, 0]",🦀🦀🦀🦀🦀🦀🦀🦀🦀I’m bad at this game🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀🦀


In [13]:
df = df[df['emotions'].apply(lambda x: len(x) <= 3)]

In [14]:
df['emotions'].value_counts()

emotions
[happiness]                    8456
[happiness, neutral]           4926
[anger, neutral]               4254
[neutral]                      4240
[anger]                        2274
                               ... 
[disgust, surprise, fear]        11
[happiness, surprise, fear]       9
[happiness, disgust, fear]        6
[fear, anger, surprise]           6
[surprise, anger, fear]           4
Name: count, Length: 94, dtype: int64

### aaa

In [15]:
# Text Cleaning
df['Clean_Text'] = df['text'].apply(nfx.remove_userhandles)

In [16]:
df['Clean_Text'] = df['Clean_Text'].apply(nfx.remove_emojis)

In [17]:
#df['Clean_Text'] = df['Clean_Text'].apply(nfx.remove_stopwords)

In [18]:
df['Clean_Text'] = df['Clean_Text'].apply(nfx.remove_punctuations)

In [19]:
inputs = mytokenizer(df['Clean_Text'].tolist(), padding=True, truncation=True, max_length=MAX_LEN, return_tensors='pt')

In [20]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(classes=['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral'])

In [21]:
mlb

MultiLabelBinarizer(classes=['anger', 'disgust', 'fear', 'happiness', 'sadness',
                             'surprise', 'neutral'])

In [22]:
x = df['Clean_Text']
y = df['label_vector']

In [23]:
train_ratio = 0.90
#validation_ratio = 0.05
test_ratio = 0.10

from sklearn.model_selection import train_test_split
# train is now 90% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1 - train_ratio,random_state=7,stratify=y)

x_train = df['Clean_Text']
y_train = df['label_vector']

# test is now 10% of the initial data set
# validation is now 10% of the initial data set
#x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio),random_state=42,stratify=y_test) 

In [24]:
class EmotionDataset(Dataset):
    def __init__(self, texts, label_vectors, tokenizer, max_len):
        self.texts = texts
        self.label_vectors = label_vectors
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        labels = torch.tensor(self.label_vectors[item], dtype=torch.float)

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': labels
        }


In [25]:
train_dataset = EmotionDataset(texts=x_train.tolist(), label_vectors=y_train.tolist(), tokenizer=mytokenizer, max_len=MAX_LEN)
test_dataset = EmotionDataset(texts=x_test.tolist(), label_vectors=y_test.tolist(), tokenizer=mytokenizer, max_len=MAX_LEN)
#val_dataset = EmotionDataset(texts=x_val.tolist(), labels=y_val.tolist(), tokenizer=mytokenizer, max_len=MAX_LEN)

# DataLoader oluşturma
train_dataloader = DataLoader(train_dataset, batch_size=8, sampler=RandomSampler(train_dataset))
test_dataloader = DataLoader(test_dataset, batch_size=8, sampler=SequentialSampler(test_dataset))
#val_dataloader = DataLoader(val_dataset, batch_size=VALID_BATCH_SIZE, sampler=SequentialSampler(val_dataset))

In [26]:
emotion_labels = ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral']

In [27]:
#model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = len(emotion_labels), problem_type="multi_label_classification")
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = len(emotion_labels), problem_type="multi_label_classification")
#model = RobertaForSequenceClassification.from_pretrained('roberta-base',num_labels=len(emotion_labels), problem_type="multi_label_classification")
# Modeli cihazımıza gönderelim (GPU varsa CUDA'ya, yoksa CPU'ya)
model = model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
def evaluate_model_in(model, dataloader):
    model.eval()
    total_correct = 0
    total_samples = 0
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device).float()

            # Model çıktısı
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item()

            # Sigmoid uygula ve en yüksek olasılık tahminini al
            probs = sigmoid(logits)
            preds = torch.argmax(probs, dim=1)

            # "En az bir doğru eşleşme" kontrolü
            for i in range(len(preds)):
                if labels[i][preds[i]] == 1:
                    total_correct += 1
                total_samples += 1

    avg_loss = total_loss / len(dataloader)
    acc = (total_correct / total_samples) * 100
    print(f"Validation Loss: {avg_loss:.4f}, Validation Accuracy: {acc:.2f}%")


In [29]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
from torch.nn.functional import sigmoid

def train_model(model, train_dataloader, val_dataloader, epochs, lr):
    optimizer = AdamW(model.parameters(), lr=lr, weight_decay=0)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

    model = model.to(device)
   
    epoch_count = 1
    printed_hello1 = False
    printed_hello2 = False
    printed_hello3 = False
    printed_hello4 = False
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        total_correct = 0
        total_samples = 0

        loop = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}", ncols=100)

        for step, batch in enumerate(loop):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device).float()

            optimizer.zero_grad()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item()

            # ---- MULTI-LABEL SINGLE PREDICT ACCURACY ----
            probs = sigmoid(logits)
            preds = torch.argmax(probs, dim=1)

            # accuracy: en az bir doğru varsa kabul
            for i in range(len(preds)):
                if labels[i][preds[i]] == 1:
                    total_correct += 1
                total_samples += 1

            
            # Geri yayılım
            loss.backward()
            optimizer.step()

            # tqdm barını güncelle 
            acc = (total_correct / total_samples) * 100
            loop.set_postfix(loss=loss.item(), accuracy=acc)

            # Eğitim çubuğunda gösterilecek değerleri güncelleme
            accuracy = (total_correct / total_samples) * 100
            loop.set_postfix(loss=loss.item(), accuracy=accuracy)  # Progress bar'ın altında gösterilecek bilgileri güncelliyoruz


            if step / len(train_dataloader) >= 0.10 and not printed_hello3:
                #evaluate_model_in(model, val_dataloader)  # Burada normal çıktı olarak yazdırıyoruz
                printed_hello3 = True
            if step / len(train_dataloader) >= 0.20 and not printed_hello4:
                #evaluate_model_in(model, val_dataloader)  # Burada normal çıktı olarak yazdırıyoruz
                printed_hello4 = True
            if step / len(train_dataloader) >= 0.33 and not printed_hello1:
                #if epoch == 0:
                #evaluate_model_in(model, val_dataloader)  # Burada normal çıktı olarak yazdırıyoruz
                printed_hello1 = True

            if step / len(train_dataloader) >= 0.50 and not printed_hello2:
                evaluate_model_in(model, val_dataloader)  # Burada normal çıktı olarak yazdırıyoruz
                printed_hello2 = True
                
            #if step / len(train_dataloader) >= 0.33 and epoch>0:
            #    print(f"Epoch {epoch} tamamlandı, %33'e ulaşıldı. Yeni epoch başlıyor...")
            #    break
            
            
            torch.cuda.empty_cache()

        scheduler.step()

        avg_loss = total_loss / len(train_dataloader)
        epoch_acc = (total_correct / total_samples) * 100
        
        print(f'\nEpoch {epoch + 1}/{epochs} => Loss: {avg_loss:.4f}, Accuracy: {epoch_acc:.2f}%')
        
        printed_hello1 = False
        printed_hello2 = False
        printed_hello3 = False
        printed_hello4 = False
        epoch_count = epoch_count + 1
        
        evaluate_model_in(model, val_dataloader)

In [30]:
mlb

MultiLabelBinarizer(classes=['anger', 'disgust', 'fear', 'happiness', 'sadness',
                             'surprise', 'neutral'])

In [31]:
import torch
torch.cuda.empty_cache()

In [33]:
# Modeli eğitiyoruz
train_model(model, train_dataloader, test_dataloader, epochs=3, lr=LEARNING_RATE)

Epoch 1/3:  50%|██████████          | 714/1425 [13:29<2:29:54, 12.65s/it, accuracy=78.6, loss=0.351]

Validation Loss: 0.3303, Validation Accuracy: 84.69%


Epoch 1/3: 100%|█████████████████████| 1425/1425 [27:29<00:00,  1.16s/it, accuracy=81.9, loss=0.335]



Epoch 1/3 => Loss: 0.3546, Accuracy: 81.93%
Validation Loss: 0.3011, Validation Accuracy: 88.22%


Epoch 2/3:  50%|██████████          | 714/1425 [17:41<2:37:28, 13.29s/it, accuracy=88.2, loss=0.266]

...


Epoch 2/3 => Loss: 0.2980, Accuracy: 88.70%
Validation Loss: 0.2782, Validation Accuracy: 90.37%


Epoch 3/3:  50%|██████████          | 714/1425 [17:23<6:52:22, 34.80s/it, accuracy=88.8, loss=0.378]

Validation Loss: 0.2797, Validation Accuracy: 90.41%


Epoch 3/3: 100%|█████████████████████| 1425/1425 [31:06<00:00,  1.31s/it, accuracy=89.5, loss=0.249]



Epoch 3/3 => Loss: 0.2896, Accuracy: 89.51%
Validation Loss: 0.2765, Validation Accuracy: 90.43%


In [34]:
import torch
torch.cuda.empty_cache()

In [35]:
def predict_sentence(sentence):
    # Cümleyi tokenize et
    inputs = mytokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # Modeli eval moduna al
    model.eval()
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    logits = outputs.logits
    probs = torch.sigmoid(logits)  # Multi-label olduğu için sigmoid

    
    # Olasılıkları ayarla
    for i in range(probs.size(0)):
        # class indexlerine göre düzeltmeler
        for j, class_name in enumerate(mlb.classes):
            if class_name == "neutral":
                probs[i][j] *= 0.7
            elif class_name == "happiness":
                probs[i][j] *= 0.85

    
    predicted_index = torch.argmax(probs, dim=1).item()  # En yüksek olasılığı al
    
    # Sınıf adlarını çözmek için
    class_names = mlb.classes
    predicted_class_name = class_names[predicted_index]
    
    return predicted_class_name, probs.squeeze().tolist()

In [36]:
def predict_and_display_probabilities(sentence):
    predicted_class_name, probabilities = predict_sentence(sentence)
    
    print(f"\nPredicted Emotion: **{predicted_class_name}**\n")
    
    
    # Sınıf isimleri ile olasılıkları eşleştir
    class_probabilities = list(zip(mlb.classes, probabilities))  
    
    print("Probabilities for each emotion class:")
    
    # Olasılıkları büyükten küçüğe sıralama
    sorted_class_probabilities = sorted(class_probabilities, key=lambda x: x[1], reverse=True)
    # Sınıf olasılıklarını yazdır
    for class_name, prob in sorted_class_probabilities:
        print(f"{class_name}: {prob*100:.2f}%")

In [38]:
sentence = "I remember her sinking beneath the waters, and then, and then... And so it was, that love was the gravest sin of all."

In [40]:
sentence = "I think I will miss you forever, like the stars miss the sun in the morning sky."

In [50]:
sentence = "Don't get me wrong, this is a wonderful advancement that could help a lot of people, but at the same time, it feels like the backstory to a dystopian novel."

In [40]:
sentence = "This is both incredible and terrifying …. Just think these are the things they are telling us about 😮"

In [54]:
sentence = "When he says tissue damage he really means brain damage, you have the possibility of losing more capabilities after procedure..... Yeah this is a pass, thanks"

In [56]:
sentence = "this is a pass no thanks"

In [109]:
sentence = "This is getting out of hand, STOP THE CHIPPING, these guys want to control us with these things"

In [60]:
sentence = "I really loved her, but she left."

In [62]:
sentence = "I love you my darling."

In [111]:
sentence = "I don't need any thing artificial connected to my brain. The body is it's own intelligence."

In [115]:
sentence = "The fact that most people in these comments are approving of this type of technology is insane to me. It blows my mind how far tech has overtaken our lives."

In [119]:
sentence = "thank you so much"

In [117]:
sentence = "Why did you do that?"

In [41]:
predict_and_display_probabilities(sentence)


Predicted Emotion: **sadness**

Probabilities for each emotion class:
sadness: 91.34%
happiness: 30.29%
surprise: 20.75%
neutral: 20.57%
fear: 4.76%
anger: 4.30%
disgust: 1.69%


In [42]:
# Modeli kaydetme
model.save_pretrained('./modelb')
# Tokenizer'ı kaydetme
mytokenizer.save_pretrained('./tokenb')

('./tokenr2\\tokenizer_config.json',
 './tokenr2\\special_tokens_map.json',
 './tokenr2\\vocab.json',
 './tokenr2\\merges.txt',
 './tokenr2\\added_tokens.json')

In [43]:
# Kaydettikten sonra tekrar yüklemek için:

# Modeli yükleme
#model1 = DistilBertForSequenceClassification.from_pretrained('./modeld1')
model1 = BertForSequenceClassification.from_pretrained('./modelb')
#model1 = RobertaForSequenceClassification.from_pretrained('./modelr2')
# Tokenizer'ı yükleme
#mytokenizer1 = DistilBertTokenizer.from_pretrained('./tokend1')
mytokenizer1 = BertTokenizer.from_pretrained('./tokenb')
#mytokenizer1 = RobertaTokenizer.from_pretrained('./tokenr2')